# Training symboloc regression models
This notebook illustrates how to use  scripts developed to train symbolic regression models. For more information refer to the [main manuscript](xxx)  

To run the following examples, ensure you have installed the right dependencies in your environment:
* numpy
* pandas
* sympy
* scikit-learn
* autofeat

In [1]:
import numpy as np
import pandas as pd
import workflows as wf

## Training data

For this example we generate artifical training data from a arbitrarily chosen expression. In your application, however, you will replace this function for a function opening an existing dataset from, e.g. a pickle file, or a csv. Your training data should be in a pandas dataframe of shape (n samples) x (m predictors + target).

In [61]:
# -------------- TRAINING DATA -------------------

def generate_training_data():
    """Example function to generate toy training set"""

    npoints = 50
    t_coords = np.linspace(243, 330, npoints) #temperature samples
    c_coords = np.linspace(0.1, 3, npoints) #concentration samples
    b1, b2 = 1, -2e-3 #coefficients

    #training dataframe
    train_data = pd.DataFrame(data = np.array([np.repeat(t_coords,npoints**2), np.hstack((npoints**2)*(c_coords,))]).T, 
                            columns=['T', 'c'])

    #apply expression to be found via symbolic regression: k = b1*c*t**1.5 + b2*t*c**1.5
    train_data['k'] = b1*train_data['c']*train_data['T'].pow(0.5) + b2*train_data['T']*train_data['c'].pow(1.5)
    
    return train_data


In [63]:
# --------------  HYPER-PARAMETERS FEATURE GENERATION -------------------

#how to scale data. Supported 'standard_nomean', 'standard', 'none'
SCALING_TYPE = 'standard_nomean' 

#whether to leave intercept to vary freely (True) or constrain its value to y0 = 0 (False).
FIT_INTERCEPT = False

# Autofeat hyperparameter.
# number of times features are combined to obtain ever more complex features.
# example FEATENG_STEPS = 3 with sqrt transformations will find terms like sqrt(sqrt(sqrt(x)))
FEATENG_STEPS = 5

# Autofeat hyperparameter.
# Units of predictors. Keys must match column names in dataframe. 
# Ignored predictors are assumed to be dimensionless.
UNITS = {"T": "1/K",
        "c": "mol/kg"}

# Autofeat hyperparameter.
# Number of iterations for filtering out generated features.
FEATSEL_RUNS = 3

# Autofeat hyperparameter.
# Set of non-linear transformations to be applied to initial predictors.
TRANSFORMATIONS = ["sqrt", "sqrt"]


# --------------  HYPER-PARAMETERS FEATURE SELECTION -------------------

# n-standard deviations criterion to choose optimal alpha from Cross Validation. 
# Higher STD_ALPHA lead to sparser solutions.
STD_ALPHA = 1 

#t-statistic rejection threshold. Coefficients with t-statistic < REJECTION_THR are rejected.
REJECTION_THR = 2 


In [64]:
# ---------------- INSTANTIATE WORKFLOW --------------------------------------
training_data = generate_training_data()

workflow = wf.WorkflowAF(feateng_steps = FEATENG_STEPS,
                        units =  UNITS,
                        featsel_runs = FEATSEL_RUNS,
                        transformations = TRANSFORMATIONS,
                        xtrain = training_data[['c','T']], 
                        ytrain = training_data['k'], 
                        scaling_type = SCALING_TYPE,
                        stdalpha =  STD_ALPHA, 
                        rejection_thresshold = REJECTION_THR, 
                        fit_intercept = FIT_INTERCEPT) 

# ---------------- RUN TRAINING WORKFLOW --------------------------------------
trained_workflow = workflow.run_workflow()

[AutoFeat] Warning: This just calls fit_transform() but does not return the transformed dataframe.
[AutoFeat] It is much more efficient to call fit_transform() instead of fit() and transform()!
[AutoFeat] Applying the Pi Theorem
[AutoFeat] The 5 step feature engineering process could generate up to 1354 features.
[AutoFeat] With 125000 data points this new feature matrix would use about 0.68 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 2 transformed features from 2 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 6 feature combinations from 6 original feature tuples - done.
[feateng] Step 3: transformation of new features
[feateng] Generated 6 transformed features from 6 original features - done.
[feateng] Step 4: combining old and new features
[feateng] Generated 36 feature combinations from 48 original feature tuples - done.
[feateng] Step 5: combining new features
[feateng] Generated 58 feature co

In [65]:
# ------------------------- INSPECT SYMBOLIC MODEL --------------------------
trained_workflow.coeff_table

,mean,stdev,coeff,coeff stdev,coeff |t|,coeff_corr
c,1.550000,0.854075,11.330666,0.001046,10834.582018,13.266590
T**(3/2)*sqrt(T*c),97951.021798,36756.952537,2.445561,0.001046,2338.488253,0.000067
T,286.500000,25.622259,-0.186925,0.001046,178.741006,-0.007295


In [66]:
trained_workflow.eqn

6.653328793234911e-5*T**(3/2)*sqrt(T*c) - 0.0072954162164353764*T + 13.266589527514514*c

In [67]:
# ------------------------- EVALUATE SYMBOLIC MODEL --------------------------
import sklearn.metrics as skmetrics 

y_real = training_data['k']
y_hat = trained_workflow.predict(x = training_data[['c','T']])

print('mse: {}'.format(skmetrics.mean_squared_error(y_real, y_hat)))
print('r2: {}'.format(skmetrics.r2_score(y_real, y_hat)))
print('mape: {}'.format(skmetrics.mean_absolute_percentage_error(y_real, y_hat)))

mse: 0.1367177906453799
r2: 0.9992615352216608
mape: 0.02601023550725349
